In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [2]:
def find_nearby_rows(df, distance_threshold=6):
    """
    각 행의 위도 및 경도를 비교하고, 거리가 지정된 임계값(km) 이내인 행을 찾는 함수.

    Args:
        df (pd.DataFrame): 위도, 경도 및 ID 열을 포함하는 df
        distance_threshold (float): km 단위의 거리 임계값 (설정값값: 6)

    Returns:
        pd.DataFrame: 각 행에 대해 거리가 임계값 이내인 다른 행의 ID를 포함하는 df
    """

    n_rows = len(df)
    nearby_ids = []

    for i in range(n_rows):
        row1 = df.iloc[i]
        lat1, lon1 = row1['경도'], row1['위도']
        nearby_rows_for_i = []

        for j in range(n_rows):
            if i == j:  # 동일한 행은 비교하지 X
                continue

            row2 = df.iloc[j]
            lat2, lon2 = row2['경도'], row2['위도']

            # geopy의 geodesic 함수 사용, 두 지점 간의 거리를 계산
            distance = geodesic((lat1, lon1), (lat2, lon2)).km

            if distance <= distance_threshold:
                nearby_rows_for_i.append(row2['ID'])

        nearby_ids.append(nearby_rows_for_i)

    # result df 생성
    result_df = pd.DataFrame(index=df.index)

    for k in range(len(nearby_ids)):
      for l in range(len(nearby_ids[k])):
        result_df.loc[k, f'nearby_id_{l+1}'] = nearby_ids[k][l]

    return result_df


# 입력데이터
df = pd.read_csv("tree_result_updated.csv")


result = find_nearby_rows(df)
print(result)

KeyboardInterrupt: 

In [ ]:
result.to_csv("nearby_id_under6km_tree.csv")

In [ ]:
links = []
for i in range(len(df)):
    for j in range(1, len(result.columns) + 1):  # nearby_id 열 순회
        nearby_id = result.loc[i, f'nearby_id_{j}']
        if pd.notna(nearby_id):  # NaN 값이 아닌 경우에만 링크 생성
            links.append({'Source': df.loc[i, 'ID'], 'Target': nearby_id})

links_df = pd.DataFrame(links)
print("\n링크 데이터:\n", links_df)

In [ ]:
links_df.to_csv("links_cer.csv")